<a href="https://colab.research.google.com/github/Decko048/CIMSpark/blob/spark-2.3.0/EDA_Printing_Performance2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade plotly

     |████████████████████████████████| 26.5 MB 91.5 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [2]:
import pandas as pd
import numpy as np
import statistics
import datetime

from scipy import stats
from scipy.stats import skew, kurtosis

import seaborn as sns 
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
xls = pd.ExcelFile('/content/drive/My Drive/Work/2021 Raw Dataset Detail Speed Notif Printing.xlsx')

FileNotFoundError: ignored

In [ ]:
# opening 1st sheet
d1 = pd.read_excel(xls,'01-03', skiprows=2)

In [ ]:
d1.head()

,McCode,OrderID,JobName,NoJI,WidthCyl,Date,Weekly,Hour,Films,Type,Target,LowCenterLine,interval,Speed,Under_lcl,Notif,OnTheSpot,MatteAgent,Weight,QtyOkMtr,BdCode,RunTime,Downtime
0,PR01,NaN,NaN,NaN,NaN,2021-01-01,1- W1,0:00,NaN,NaN,0,0,0,0,N,N,N,N,0.2,0,PF,0,1
1,PR01,NaN,NaN,NaN,NaN,2021-01-01,1- W1,0:01,NaN,NaN,0,0,0,0,N,N,N,N,0.2,0,US,0,1
2,PR01,NaN,NaN,NaN,NaN,2021-01-01,1- W1,0:02,NaN,NaN,0,0,0,0,N,N,N,N,0.2,0,US,0,1
3,PR01,NaN,NaN,NaN,NaN,2021-01-01,1- W1,0:03,NaN,NaN,0,0,0,0,N,N,N,N,0.2,0,US,0,1
4,PR01,NaN,NaN,NaN,NaN,2021-01-01,1- W1,0:04,NaN,NaN,0,0,0,0,N,N,N,N,0.2,0,US,0,1


# **Preprocessing?**

In [ ]:
# slicing the data to only one machine
pr01 = d1[d1['McCode'] == 'PR01']
pr01

,McCode,OrderID,JobName,NoJI,WidthCyl,Date,Weekly,Hour,Films,Type,Target,LowCenterLine,interval,Speed,Under_lcl,Notif,OnTheSpot,MatteAgent,Weight,QtyOkMtr,BdCode,RunTime,Downtime
0,PR01,NaN,NaN,NaN,NaN,2021-01-01,1- W1,0:00,NaN,NaN,0,0,0,0,N,N,N,N,0.2,0,PF,0,1
1,PR01,NaN,NaN,NaN,NaN,2021-01-01,1- W1,0:01,NaN,NaN,0,0,0,0,N,N,N,N,0.2,0,US,0,1
2,PR01,NaN,NaN,NaN,NaN,2021-01-01,1- W1,0:02,NaN,NaN,0,0,0,0,N,N,N,N,0.2,0,US,0,1
3,PR01,NaN,NaN,NaN,NaN,2021-01-01,1- W1,0:03,NaN,NaN,0,0,0,0,N,N,N,N,0.2,0,US,0,1
4,PR01,NaN,NaN,NaN,NaN,2021-01-01,1- W1,0:04,NaN,NaN,0,0,0,0,N,N,N,N,0.2,0,US,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519221,PR01,NaN,NaN,NaN,NaN,2021-03-31,3- W5,23:50,NaN,NaN,0,0,0,226,N,N,N,N,0.2,0,PF,0,1
519222,PR01,NaN,NaN,NaN,NaN,2021-03-31,3- W5,23:51,NaN,NaN,0,0,0,226,N,N,N,N,0.2,0,PF,0,1
519223,PR01,NaN,NaN,NaN,NaN,2021-03-31,3- W5,23:52,NaN,NaN,0,0,0,226,N,N,N,N,0.2,0,PF,0,1
519224,PR01,NaN,NaN,NaN,NaN,2021-03-31,3- W5,23:53,NaN,NaN,0,0,0,226,N,N,N,N,0.2,0,PF,0,1


In [ ]:
# drop unnecessary colums
pr01 = pr01.drop(columns= ['McCode', 'OrderID', 'JobName', 'NoJI','WidthCyl','Under_lcl','Weight','MatteAgent','Type','interval'], axis=1)
pr01

,Date,Weekly,Hour,Films,Target,LowCenterLine,Speed,Notif,OnTheSpot,QtyOkMtr,BdCode,RunTime,Downtime
0,2021-01-01,1- W1,0:00,NaN,0,0,0,N,N,0,PF,0,1
1,2021-01-01,1- W1,0:01,NaN,0,0,0,N,N,0,US,0,1
2,2021-01-01,1- W1,0:02,NaN,0,0,0,N,N,0,US,0,1
3,2021-01-01,1- W1,0:03,NaN,0,0,0,N,N,0,US,0,1
4,2021-01-01,1- W1,0:04,NaN,0,0,0,N,N,0,US,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
519221,2021-03-31,3- W5,23:50,NaN,0,0,226,N,N,0,PF,0,1
519222,2021-03-31,3- W5,23:51,NaN,0,0,226,N,N,0,PF,0,1
519223,2021-03-31,3- W5,23:52,NaN,0,0,226,N,N,0,PF,0,1
519224,2021-03-31,3- W5,23:53,NaN,0,0,226,N,N,0,PF,0,1


In [ ]:
# change the date and hour column type
pr01['Date']= pd.to_datetime(pr01['Date'])
pr01['Hour']= pd.to_datetime(pr01['Hour'])
pr01.dtypes

Date             datetime64[ns]
Weekly                   object
Hour             datetime64[ns]
Films                    object
Target                    int64
LowCenterLine             int64
Speed                     int64
Notif                    object
OnTheSpot                object
QtyOkMtr                  int64
BdCode                   object
RunTime                   int64
Downtime                  int64
dtype: object

In [ ]:
# slice the dataset to January
pr01_jan = pr01[pr01['Date'].dt.month == 1]
pr01_jan

,Date,Weekly,Hour,Films,Target,LowCenterLine,Speed,Notif,OnTheSpot,QtyOkMtr,BdCode,RunTime,Downtime
0,2021-01-01,1- W1,2022-01-17 00:00:00,NaN,0,0,0,N,N,0,PF,0,1
1,2021-01-01,1- W1,2022-01-17 00:01:00,NaN,0,0,0,N,N,0,US,0,1
2,2021-01-01,1- W1,2022-01-17 00:02:00,NaN,0,0,0,N,N,0,US,0,1
3,2021-01-01,1- W1,2022-01-17 00:03:00,NaN,0,0,0,N,N,0,US,0,1
4,2021-01-01,1- W1,2022-01-17 00:04:00,NaN,0,0,0,N,N,0,US,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43564,2021-01-31,1- W5,2022-01-17 23:50:00,PET,230,219,25,N,N,84000,PF,0,1
43565,2021-01-31,1- W5,2022-01-17 23:51:00,PET,230,219,25,N,N,84000,PF,0,1
43566,2021-01-31,1- W5,2022-01-17 23:52:00,PET,230,219,25,N,N,84000,PF,0,1
43567,2021-01-31,1- W5,2022-01-17 23:53:00,PET,230,219,35,N,N,84000,PF,0,1


In [ ]:
# check the percentage of missing values
def missing_values(data):
    total = data.isnull().sum()
    percent = data.isnull().mean()*100
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_values(pr01_jan)

,Total,Percent
Date,0,0.000000
Weekly,0,0.000000
Hour,0,0.000000
Films,4273,9.807432
Target,0,0.000000
LowCenterLine,0,0.000000
Speed,0,0.000000
Notif,0,0.000000
OnTheSpot,0,0.000000
QtyOkMtr,0,0.000000


In [ ]:
# slicing the dataset to only "RUN" Condition
pr01_run_jan = pr01_jan[pr01_jan['BdCode'] == 'RUN'].drop(columns='Downtime')
pr01_run_jan

,Date,Weekly,Hour,Films,Target,LowCenterLine,Speed,Notif,OnTheSpot,QtyOkMtr,BdCode,RunTime
1987,2021-01-02,1- W1,2022-01-17 09:23:00,PET,200,190,180,N,N,12000,RUN,1
1988,2021-01-02,1- W1,2022-01-17 09:24:00,PET,200,190,180,N,N,12000,RUN,1
1989,2021-01-02,1- W1,2022-01-17 09:25:00,PET,200,190,180,N,N,12000,RUN,1
1990,2021-01-02,1- W1,2022-01-17 09:26:00,PET,200,190,181,Y,N,12000,RUN,1
1991,2021-01-02,1- W1,2022-01-17 09:27:00,PET,200,190,181,Y,N,12000,RUN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
41381,2021-01-30,1- W5,2022-01-17 23:45:00,NaN,0,0,186,N,N,0,RUN,1
41382,2021-01-30,1- W5,2022-01-17 23:46:00,NaN,0,0,186,N,N,0,RUN,1
41383,2021-01-30,1- W5,2022-01-17 23:47:00,NaN,0,0,186,N,N,0,RUN,1
41384,2021-01-30,1- W5,2022-01-17 23:48:00,NaN,0,0,200,N,N,0,RUN,1


# **Exploratory Data Analysis**

In [ ]:
# summary of the runtime data in January
pr01_run_jan.drop(columns = ['RunTime']).describe().T

,count,mean,std,min,25%,50%,75%,max
Target,23063.0,207.629103,47.057811,0.0,200.0,220.0,230.0,230.0
LowCenterLine,23063.0,197.415991,44.785505,0.0,190.0,209.0,219.0,219.0
Speed,23063.0,219.269913,43.335901,0.0,211.0,231.0,241.0,302.0
QtyOkMtr,23063.0,139679.443264,131331.548684,0.0,38000.0,104000.0,216000.0,560000.0


In [ ]:
# calculating mean and standard deviation
mean = pr01_run_jan['Speed'].mean()
std = pr01_run_jan['Speed'].std(ddof=0) # stdev.s, for stdev.p use ddof=1 as argument
var = pr01_run_jan['Speed'].var()
skew =  pr01_run_jan['Speed'].skew()
kurt =  pr01_run_jan['Speed'].kurt()

print('Mean: ', mean)
print('Std. Dev: ', std)
print('Sample Variance: ', var)
print('Skewness: ', skew)
print('Kurtosis: ', kurt)

Mean:  219.26991284741794
Std. Dev:  43.33496123675944
Sample Variance:  1878.0002945331219
Skewness:  -3.5663191651373953
Kurtosis:  15.594609195252264


In [ ]:
# runtime and downtime distributions in January 
runtime = pr01_jan['BdCode'].value_counts().rename_axis('BdCode').reset_index(name='Minutes')            
runtime

,BdCode,Minutes
0,RUN,23063
1,CO1,5747
2,US,3257
3,PF,3215
4,OTS,3049
5,CO2,1484
6,CO3,1303
7,MQ,1077
8,PM,618
9,BD,462


In [ ]:
# BdCode Distribution for PR01 in January
fig = px.pie(runtime, values='Minutes', names='BdCode')
fig.update_layout(title="Runtime and Downtime Distribution of Machine PR01 in January",title_x=0.5)
fig.show()

In [ ]:
# bdcode distributions per each film in January
pr01_jan_bdcode_film = pr01_jan.groupby(by =['BdCode','Films'])['RunTime'].count().to_frame().reset_index().rename(columns={'BdCode':'BdCode','Films':'Films','RunTime':'Minutes'})
pr01_jan_bdcode_film

,BdCode,Films,Minutes
0,BD,OPP,301
1,BD,PET,161
2,CO1,CPP,83
3,CO1,NYL,1309
4,CO1,OPP,1609
5,CO1,PET,2372
6,CO2,NYL,365
7,CO2,OPP,419
8,CO2,PET,636
9,CO3,CPP,47


In [ ]:
# plotting bdcode distribution per each film in January
fig = px.bar(pr01_jan_bdcode_film, x="Films", y="Minutes", color="BdCode",text_auto=True)
fig.update_layout(title="Runtime and Downtime of PR01 per Film Type",title_x=0.5, height=700)
fig.show()

In [ ]:
# avg running speed per film on machine PR01
pr01_run_jan_speed_film = pr01_run_jan.groupby([pr01_jan['Films']])['Speed'].mean().rename_axis('Films').reset_index().rename(columns={'Speed':'Avg Speed'})
pr01_run_jan_speed_film

,Films,Avg Speed
0,CPP,194.368132
1,NYL,216.120716
2,OPP,214.479185
3,PET,225.474992


In [ ]:
# total runtime per film on machine PR01
pr01_run_jan_time_film = pr01_jan.groupby([pr01_jan['Films']])['RunTime'].sum().rename_axis('Films').reset_index() 
pr01_run_jan_time_film['Downtime'] = pr01_jan.groupby([pr01_jan['Films']])['Downtime'].sum().rename_axis('Films').reset_index().drop(columns='Films')
pr01_run_jan_time_film['RunTime%'] = pr01_run_jan_time_film['RunTime'] / (pr01_run_jan_time_film['RunTime'] + pr01_run_jan_time_film['Downtime']) * 100
pr01_run_jan_time_film

,Films,RunTime,Downtime,RunTime%
0,CPP,182,179,50.415512
1,NYL,3968,4104,49.157582
2,OPP,9080,4771,65.554834
3,PET,8977,8035,52.768634


In [ ]:
# output per film on machine PR01 in January
pr01_run_jan_film = pr01_run_jan_speed_film.copy()
pr01_run_jan_film['RunTime'] = pr01_run_jan_time_film['RunTime']
pr01_run_jan_film['Downtime'] = pr01_run_jan_time_film['Downtime']
pr01_run_jan_film['RunTime%'] = pr01_run_jan_time_film['RunTime%']
pr01_run_jan_film['Output'] = pr01_run_jan_film['Avg Speed'] * pr01_run_jan_film['RunTime']
pr01_run_jan_film = pr01_run_jan_film.sort_values('Output', ascending=False).reset_index().drop(columns='index')
pr01_run_jan_film

,Films,Avg Speed,RunTime,Downtime,RunTime%,Output
0,PET,225.474992,8977,8035,52.768634,2024089.0
1,OPP,214.479185,9080,4771,65.554834,1947471.0
2,NYL,216.120716,3968,4104,49.157582,857567.0
3,CPP,194.368132,182,179,50.415512,35375.0


In [ ]:
# plotting output per film for PR01 in January
fig = px.bar(pr01_run_jan_film, x='Films', y='Output', color='Output',text_auto=True,color_continuous_scale=px.colors.sequential.Viridis)
fig.update_layout(title_text='PR01 Running Output in January Based on Film',xaxis_title="Film",yaxis_title="Output",title_x=0.5)
fig.show()

In [ ]:
# sum of the runtime and dowtime per day in January
runtime_jan_sum = pr01_run_jan.groupby([pr01_run_jan['Date'].dt.date]).sum().rename_axis('Date').reset_index() 
runtime_jan_sum = runtime_jan_sum[['Date','RunTime']].copy()
runtime_jan_sum['Downtime'] = pr01_jan.groupby([pr01_jan['Date'].dt.date]).sum().rename_axis('Date').reset_index().loc[:,'Downtime']
runtime_jan_sum['RunTime%'] = runtime_jan_sum['RunTime'] / (runtime_jan_sum['RunTime'] + runtime_jan_sum['Downtime']) * 100

# calculating mean and standard deviation
mean = runtime_jan_sum['RunTime'].mean()
std = runtime_jan_sum['RunTime'].std(ddof=0) # stdev.s, for stdev.p use ddof=1 as argument
var = runtime_jan_sum['RunTime'].var()
skew =  runtime_jan_sum['RunTime'].skew()
kurt =  runtime_jan_sum['RunTime'].kurt()

print('Mean: ', mean)
print('Std. Dev: ', std)
print('Sample Variance: ', var)
print('Skewness: ', skew)
print('Kurtosis: ', kurt)

# calculating cdf and pdf
# cdf = stats.norm.cdf(runtime_jan_sum['RunTime'], mean, std)
# pdf = stats.norm.pdf(runtime_jan_sum['RunTime'], mean, std)

runtime_jan_sum

Mean:  823.6785714285714
Std. Dev:  259.8617838076108
Sample Variance:  70029.18915343913
Skewness:  -0.1694199850632625
Kurtosis:  -0.6248534378079986


,Date,RunTime,Downtime,RunTime%
0,2021-01-02,524,1427,26.858022
1,2021-01-03,896,903,49.805447
2,2021-01-04,1172,526,69.022379
3,2021-01-05,219,254,46.300211
4,2021-01-06,829,1198,40.897879
5,2021-01-07,484,590,45.065177
6,2021-01-08,523,939,35.772914
7,2021-01-09,1126,886,55.964215
8,2021-01-10,916,298,75.453048
9,2021-01-11,758,491,60.688551


In [ ]:
# average datas per day in January based on "RUN" condition
runperdate_jan_mean = pr01_run_jan.groupby([pr01_run_jan['Date'].dt.date]).mean().rename_axis('Date').reset_index()
runperdate_jan_mean = runperdate_jan_mean.drop(columns=['RunTime'])
runperdate_jan_mean

,Date,Target,LowCenterLine,Speed,QtyOkMtr
0,2021-01-02,197.786260,188.057252,212.585878,24366.412214
1,2021-01-03,211.808036,201.573661,213.797991,95185.267857
2,2021-01-04,225.486348,214.702218,233.936860,138542.662116
3,2021-01-05,164.337900,156.383562,217.890411,91123.287671
4,2021-01-06,219.553679,208.975875,223.805790,84332.931242
5,2021-01-07,187.024793,177.818182,211.103306,43097.107438
6,2021-01-08,194.493308,184.768642,216.978967,91609.942639
7,2021-01-09,213.534636,202.857904,240.126110,210952.042629
8,2021-01-10,202.554585,192.522926,211.601528,81543.668122
9,2021-01-11,208.786280,198.633245,228.192612,257663.588391


In [ ]:
runperdate_jan = runperdate_jan_mean[['Date', 'Speed']].rename(columns = {'Speed':'Avg Speed'})
runperdate_jan['RunTime'] = runtime_jan_sum[['RunTime']].copy()
runperdate_jan['Output'] = runperdate_jan['Avg Speed'] * runperdate_jan['RunTime']
runperdate_jan

,Date,Avg Speed,RunTime,Output
0,2021-01-02,212.585878,524,111395.0
1,2021-01-03,213.797991,896,191563.0
2,2021-01-04,233.936860,1172,274174.0
3,2021-01-05,217.890411,219,47718.0
4,2021-01-06,223.805790,829,185535.0
5,2021-01-07,211.103306,484,102174.0
6,2021-01-08,216.978967,523,113480.0
7,2021-01-09,240.126110,1126,270382.0
8,2021-01-10,211.601528,916,193827.0
9,2021-01-11,228.192612,758,172970.0


In [ ]:
# plotting the output PR01 per Day in January
fig = px.line(runperdate_jan, x='Date', y='Output', markers=True)
fig.update_layout(title="PR01 Output per Day in January",title_x=0.5)
fig.show()

In [ ]:
# plotting the average LowCenterLine and Speed of PR01 per Day in January
figure=pd.melt(runperdate_jan_mean, id_vars=['Date'], value_vars=['Speed','LowCenterLine'])

fig = px.line(figure, x='Date', y='value',color='variable', markers=True)
fig.update_layout(title="Average LowCenterLine and Speed of PR01 per Day in January",title_x=0.5)
fig.show()

In [ ]:
data= {}
for i in range (2,31):
   data[i] = pr01_jan[pr01_jan['Date'].dt.day == i]

In [ ]:
# total run time per BdCode on 19th January
runtime19 = data[19]['BdCode'].value_counts().rename_axis('BdCode').reset_index(name='Minutes')                
runtime19

,BdCode,Minutes
0,RUN,708
1,PF,253
2,BD,156
3,MQ,44
4,US,38
5,MC,35
6,CO2,19
7,CO1,17
8,CO3,7


In [ ]:
# runtime vs downtime distribution on January 19th, the PF condition seems to be higher           
fig = px.pie(runtime19, values='Minutes', names='BdCode')
fig.update_layout(title="Runtime vs Downtime of Machine PR01 on 19th January",title_x=0.5)
fig.show()

In [ ]:
# average data on 19th January based on BD Code
bdcode19 = data[19].groupby([data[19]['BdCode']]).mean().rename_axis('BdCode').reset_index()
bdcode19

,BdCode,Target,LowCenterLine,Speed,QtyOkMtr,RunTime,Downtime
0,BD,220.000000,209.000000,216.685897,248000.000000,0.0,1.0
1,CO1,220.000000,209.000000,221.000000,248000.000000,0.0,1.0
2,CO2,220.000000,209.000000,180.631579,248000.000000,0.0,1.0
3,CO3,220.000000,209.000000,20.857143,248000.000000,0.0,1.0
4,MC,220.000000,209.000000,214.685714,248000.000000,0.0,1.0
5,MQ,220.000000,209.000000,175.363636,248000.000000,0.0,1.0
6,PF,219.130435,208.173913,114.094862,247019.762846,0.0,1.0
7,RUN,213.474576,202.800847,120.505650,240644.067797,1.0,0.0
8,US,220.000000,209.000000,182.552632,248000.000000,0.0,1.0


In [ ]:
# Plotting the average speed based on BD Code
# average "RUN" speed on 19th January seems to be lower than the other Code
fig = px.bar(bdcode19, x='BdCode', y='Speed', color='Speed',color_continuous_scale=px.colors.sequential.Viridis)
fig.update_layout(title_text='Avg Runtime vs Downtime Speed on January 19th',xaxis_title="BdCode",yaxis_title="Speed",title_x=0.5)
fig.show()

In [ ]:
# plotting the average speed in January based on Code
# average speed in January is much faster than the average Speed on 19th January (see graph above for 19th January)
bdcode_jan = pr01_jan.groupby([pr01_jan['BdCode']]).mean().rename_axis('BdCode').reset_index()
fig = px.bar(bdcode_jan, x='BdCode', y='Speed', color='Speed',color_continuous_scale=px.colors.sequential.Viridis)
fig.update_layout(title_text='Average Speed in January Based on Breakdown Code',xaxis_title="BdCode",yaxis_title="Speed",title_x=0.5)
fig.show()

In [ ]:
runtime21 = data[21]['BdCode'].value_counts().rename_axis('BdCode').reset_index(name='Minutes')             

In [ ]:
# plotting the BD Code distribution on January 19th
# the PF condition seems to be higher
fig = px.pie(runtime21, values='Minutes', names='BdCode')
fig.update_layout(title="Breakdown of Machine PR01 on 21st January",title_x=0.5)
fig.show()

In [ ]:
# average speed on "RUN" condition on 21st January seems to be higher but still much lower than average
bdcode21 = data[21].groupby([data[21]['BdCode']]).mean().rename_axis('BdCode').reset_index() 
bdcode21

,BdCode,Target,LowCenterLine,Speed,QtyOkMtr,RunTime,Downtime
0,CO1,203.103448,193.011494,104.743295,99697.318008,0.0,1.0
1,CO2,197.090909,187.236364,139.036364,108690.909091,0.0,1.0
2,CO3,188.571429,179.142857,101.892857,43571.428571,0.0,1.0
3,MQ,198.181818,188.272727,39.363636,273090.909091,0.0,1.0
4,OTS,188.925620,179.479339,124.958678,134958.677686,0.0,1.0
5,PF,195.437956,185.729927,112.718978,47536.496350,0.0,1.0
6,RUN,197.028169,187.283099,169.946479,119132.394366,1.0,0.0


In [ ]:
# Plotting average speed on BD Code on 21st January
fig = px.bar(bdcode21, x='BdCode', y='Speed', color='Speed',color_continuous_scale=px.colors.sequential.Viridis)
fig.update_layout(title_text='Speed on January 21st Based on BD Code',xaxis_title="BdCode",yaxis_title="Speed",title_x=0.5)
fig.show()

In [ ]:
# total Whatsapp notification in January based on "RUN" condition
notif_jan_total = pr01_run_jan['Notif'].value_counts().rename_axis('Notif').reset_index(name='Count')
notif_jan_total             

,Notif,Count
0,N,20598
1,Y,2465


In [ ]:
# total notifications per day in January based on "RUN" condition
notifperday_jan = pr01_run_jan.groupby(by =['Date','Notif'])['RunTime'].count().to_frame().reset_index().rename(columns={'Date':'Date','Notif':'Notif','Runtime':'Count'})
print('Total Runtime in January: ', notifperday_jan['RunTime'].sum())
notifperday_jan

Total Runtime in January:  23063


,Date,Notif,RunTime
0,2021-01-02,N,336
1,2021-01-02,Y,188
2,2021-01-03,N,804
3,2021-01-03,Y,92
4,2021-01-04,N,1156
5,2021-01-04,Y,16
6,2021-01-05,N,219
7,2021-01-06,N,771
8,2021-01-06,Y,58
9,2021-01-07,N,395


In [ ]:
# plotting total notifications per day in January based on "RUN"
fig = px.line(notifperday_jan, x='Date', y='RunTime',color='Notif', markers=True)
fig.update_layout(title="Total Notifications of PR01 per Day in January",title_x=0.5,yaxis_title='Count')
fig.show()

In [ ]:
 notif_jan = pr01_run_jan.copy()
 notif_jan['NotifAct'] = np.where(pr01_run_jan['Speed'] < pr01_run_jan['LowCenterLine'], 'Y', 'N')

In [ ]:
# see false WA notif count (should be 'Y' but actually 'N' in the data and vice versa)
# 0 means it's false
test = notif_jan[['Date','Hour','Notif','NotifAct']].copy()
test['Comparison'] = np.where(test['Notif'] == test['NotifAct'], '1', '0')

test['Comparison'].value_counts().rename_axis('Comparison').reset_index(name='Count')

,Comparison,Count
0,1,21718
1,0,1345
